In [1]:
from s_clip_scripts.data_loader import get_custom_data
import pandas as pd
import json
import os

In [9]:
%%time
get_custom_data(None, data = 'FashionGen', preprocess_fn = None, is_train = True, cls = False, subclass = False)

CPU times: user 2min, sys: 1.39 s, total: 2min 2s
Wall time: 2min 2s


In [15]:
%%time
import h5py
cls = False
root = './data/fashiongen'
split = 'train'
h5_path = os.path.join(root, f"fashiongen_256_256_{split}.h5") # hdf5
h5_file = h5py.File(h5_path)

h5_file_index = h5_file['index']
h5_file_item_id = h5_file['input_productID']
h5_file_input_name = h5_file['input_name']
h5_file_input_desc = h5_file['input_description']
    
data = {}
for idx in range(len(h5_file_index)):
    item_id = int(h5_file_item_id[idx])
    input_name = h5_file_input_name[idx][0]
    input_desc = h5_file_input_desc[idx][0]

    if item_id in data:
        data[item_id]['image_idx'].append(idx)
    else:
        data[item_id] = dict(image_idx=[idx], input_name=input_name, input_desc=input_desc)
data = [dict({'id': item_id}, **data[item_id]) for item_id in data]

images = h5_file['input_image']

<timed exec>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)


CPU times: user 17.9 s, sys: 62.9 ms, total: 18 s
Wall time: 18 s


In [2]:
%%time
get_custom_data(None, data = 'FashionGen', preprocess_fn = None, is_train = True, cls = True, subclass = False)

CPU times: user 7.84 s, sys: 1.72 ms, total: 7.84 s
Wall time: 1min 17s


(<data_loader.FashionGen at 0x153a45832c80>,
 ['backpacks',
  'bag accessories',
  'belts & suspenders',
  'blankets',
  'boat shoes & moccasins',
  'boots',
  'briefcases',
  'clutches & pouches',
  'dresses',
  'duffle & top handle bags',
  'duffle bags',
  'espadrilles',
  'eyewear',
  'fine jewelry',
  'flats',
  'gloves',
  'hats',
  'heels',
  'jackets & coats',
  'jeans',
  'jewelry',
  'jumpsuits',
  'keychains',
  'lace ups',
  'lingerie',
  'loafers',
  'messenger bags',
  'messenger bags & satchels',
  'monkstraps',
  'pants',
  'pocket squares & tie bars',
  'pouches & document holders',
  'sandals',
  'scarves',
  'shirts',
  'shorts',
  'shoulder bags',
  'skirts',
  'sneakers',
  'socks',
  'suits & blazers',
  'sweaters',
  'swimwear',
  'ties',
  'tops',
  'tote bags',
  'travel bags',
  'underwear & loungewear'],
 [<function data_loader.get_custom_data.<locals>.<lambda>(c)>])

In [2]:
os.listdir('./data')

['sydney_captions',
 'scicap_data.zip',
 'polyvore',
 'scicap',
 'fashion200k',
 'UCM_captions.zip',
 'RSICD.zip',
 'polyvore-images.tar.gz',
 'RSICD',
 'UCM',
 'UCMerced_LandUse',
 'WHU-RS19',
 'RSSCN7',
 'AID',
 'NWPU-RESISC45',
 'kaggle_simpsons_characters.zip',
 'kaggle_simpsons_characters',
 'polyvore_outfits.zip',
 'polyvore_outfits']

In [8]:
%%time
get_custom_data(None, data = 'Fashion200k-CLS', preprocess_fn = None, is_train = True, cls = False, subclass = False)

CPU times: user 566 ms, sys: 258 ms, total: 825 ms
Wall time: 824 ms


(<s_clip_scripts.data_loader.Fashion200k at 0x1522c2b32230>,
 [],
 [<function s_clip_scripts.data_loader.get_custom_data.<locals>.<lambda>(c)>])

In [2]:
%%time
get_custom_data(None, data = 'Polyvore', preprocess_fn = None, is_train = True, cls = True, subclass = False)

CPU times: user 1.4 s, sys: 192 ms, total: 1.6 s
Wall time: 1.65 s


In [5]:
%%time
get_custom_data(None, data = 'Polyvore', preprocess_fn = None, is_train = True, cls = False, subclass = False)

CPU times: user 1.57 s, sys: 103 ms, total: 1.68 s
Wall time: 1.68 s


In [5]:
%%time
get_custom_data(None, data = 'Polyvore', preprocess_fn = None, is_train = True, cls = False, subclass = False)

/vol/tensusers5/nhollain/data_loader.py:226: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  category_dict = category_df[['ID', 'subcategory']].set_index('ID').T.to_dict('records')


CPU times: user 7min 11s, sys: 151 ms, total: 7min 11s
Wall time: 7min 11s


In [6]:
%%time
get_custom_data(None, data = 'Polyvore', preprocess_fn = None, is_train = True, cls = True, subclass = False)

/vol/tensusers5/nhollain/data_loader.py:226: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  category_dict = category_df[['ID', 'subcategory']].set_index('ID').T.to_dict('records')


CPU times: user 5min 50s, sys: 172 ms, total: 5min 50s
Wall time: 5min 51s


(<data_loader.Polyvore at 0x14f0ff8f4ca0>,
 ['accessories',
  'all-body',
  'bags',
  'bottoms',
  'hats',
  'jewellery',
  'outerwear',
  'scarves',
  'shoes',
  'sunglasses',
  'tops'],
 [<function data_loader.get_custom_data.<locals>.<lambda>(c)>])

In [3]:
# PROBLEMS WITH POLYVORE DATA: the data structure seems to be different from what the S-CLIP paper uses.
# I want to probably get the dict['items'] values for each folder and then take the images from there.
working_datasets = ['RSICD', 'UCM', 'Sydney', 'Fashion200k', 'SciCap', 'RSICD-CLS', 'UCM-CLS', 'Fashion200k-CLS', 'WHU-RS19', 'RSSCN7', 
                    'AID', 'RESISC45', 'Simpsons-Images', 'Polyvore', 'Polyvore-CLS', 'FashionGen', 'FashionGen-CLS']
# get_custom_data(args, data, preprocess_fn, is_train, cls, subclass, **data_kwargs):
for dataset in working_datasets:
    print(dataset)
    cls = 'CLS' in dataset
    # CLS version of most datasets just uses the non-CLS dataloader, but adds cls = True as argument
    if dataset != 'UCM-CLS': # Exception: UCM-CLS is a different dataloader altogether than UCM!
        dataset = dataset.replace('-CLS', '')
    get_custom_data(None, data = dataset, preprocess_fn = None, is_train = True, cls = cls, subclass = False)

RSICD
UCM
Sydney
Fashion200k
SciCap
RSICD-CLS
UCM-CLS
Fashion200k-CLS
WHU-RS19
RSSCN7
AID
RESISC45


In [1]:
get_custom_data(None, data = 'Simpsons-Images', preprocess_fn = None, is_train = True, cls = True, subclass = False)

Dataset ImageFolder
    Number of datapoints: 20933
    Root location: ./data/kaggle_simpsons_characters/simpsons_dataset

In [6]:
%%time
from tools import read_json
path = './data/polyvore_outfits/disjoint'
split_info = read_json(os.path.join(path, 'train.json'))
path = './data/polyvore_outfits/polyvore_item_metadata.json'
metadata = read_json(path)

CPU times: user 1.32 s, sys: 192 ms, total: 1.51 s
Wall time: 1.51 s


In [11]:
%%time
split_item_ids = []
for grouping in split_info:
    items = grouping['items']
    item_ids = [item['item_id'] for item in items] # Get image IDs
    split_item_ids += item_ids
split_item_ids   

CPU times: user 15 ms, sys: 0 ns, total: 15 ms
Wall time: 14.9 ms


['132621870',
 '153967122',
 '171169800',
 '162799044',
 '172538912',
 '172312529',
 '162715806',
 '171888747',
 '173096665',
 '170904692',
 '172482221',
 '180028994',
 '182218570',
 '180850592',
 '179806210',
 '181657245',
 '165695205',
 '198643069',
 '206048471',
 '204007990',
 '200709959',
 '195973920',
 '200139640',
 '156489567',
 '148693800',
 '200313980',
 '204650506',
 '208870650',
 '207173723',
 '207871410',
 '208870252',
 '199376553',
 '203551771',
 '203552462',
 '175711802',
 '210942072',
 '212839321',
 '213855157',
 '207459533',
 '192511179',
 '187317072',
 '180362994',
 '191221027',
 '173097736',
 '172685140',
 '164453126',
 '169982065',
 '169809127',
 '213007746',
 '212992927',
 '214327110',
 '207668984',
 '181900351',
 '202212715',
 '209966118',
 '70625358',
 '208664157',
 '193455194',
 '211421142',
 '161180948',
 '158857944',
 '88909845',
 '119901618',
 '107732394',
 '166542899',
 '151124575',
 '133932837',
 '161950492',
 '210326667',
 '209505560',
 '50235496',
 '2006396

In [24]:
%%time
data = []
cls = False
item_keys = set(metadata.keys())
split_item_ids = set(split_item_ids)
intersect = item_keys.intersection(split_item_ids)
for item_key in intersect:
    # if item_key not in split_item_ids: # Skip items not in the current split
    #     continue
    item = metadata[item_key]
    item_path = item_key + '.jpg'
    if cls: # For classification, we use the (sub)category as the class label
        data.append({"image_path": item_path, "class_name": item["semantic_category"]}) 
    else: # For captions, we use the fashion item's title (=name) and description as its caption
        data.append({"image_path": item_path, "sentences": item["title"] + "." + item["description"]})

CPU times: user 193 ms, sys: 0 ns, total: 193 ms
Wall time: 192 ms


In [23]:
%%time
item_keys = list(metadata.keys())
len(set(item_keys)), len(set(split_item_ids)), len(item_keys), len(split_item_ids)
intersect = set(item_keys).intersection(set(split_item_ids))
len(intersect)

CPU times: user 85.7 ms, sys: 0 ns, total: 85.7 ms
Wall time: 85.3 ms


71967

In [ ]:
# 200 ms